## 1. Libraries and Constants

In [ ]:
# Importing required libraries for building, training, and visualizing the model
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import os
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
# Setting constants for image processing and model training
IMAGE_SIZE = 250  # The size to which all images will be resized
BATCH_SIZE = 32   # Number of images per batch during training
CHANNELS = 3      # Number of color channels (RGB)
EPOCHS = 50       # Number of epochs to train the model

## 2. Data Loading and Sampling

In [ ]:
# Loading the image dataset from a directory, shuffling, and setting image size and batch size
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "Data",
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

# Extracting class names from the dataset (corresponding to folder names)
class_names = dataset.class_names

# Function to sample a fraction of the dataset (for example, 1/15th of the full dataset)
def sample_dataset(ds, fraction=1/15):
    ds_size = tf.data.experimental.cardinality(ds).numpy()  # Getting the size of the dataset
    sampled_size = int(ds_size * fraction)  # Calculating the sampled size
    ds = ds.take(sampled_size)  # Taking a subset of the dataset
    return ds

# Reducing the dataset to a fraction for faster experimentation
dataset = sample_dataset(dataset)
len(dataset)  # Check the number of batches in the dataset

## 3. Image Preview

In [ ]:
# Displaying a few example images from the dataset with their corresponding labels
plt.figure(figsize=(10,10))
for image_batch, image_label in dataset.take(1):  # Taking one batch (shuffled) from the dataset
    for i in range(12):  # Displaying 12 images
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))  # Show image
        plt.title(class_names[image_label[i]])  # Display class name as title
        plt.axis("off")  # Turn off axis

## 4. Dataset Splitting Function

In [ ]:
# Function to split the dataset into training, validation, and test sets
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=1000):
    ds_size = len(ds)  # Total size of the dataset
    
    # Shuffle the dataset if required
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    # Calculate the sizes of each subset
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    # Create the train, validation, and test datasets
    train_ds = ds.take(train_size)  # Take the first portion for training
    val_ds = ds.skip(train_size).take(val_size)  # Skip the training set and take validation set
    test_ds = ds.skip(train_size).skip(val_size)  # Skip training and validation, take the rest as test set
    
    return train_ds, val_ds, test_ds


In [ ]:
# Splitting the dataset into training, validation, and test sets
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [ ]:
# Verifying the sizes of the train, validation, and test datasets
len(train_ds), len(val_ds), len(test_ds)

## 5. Data Preprocessing

In [ ]:
# Improving data pipeline performance by caching and prefetching batches
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Preprocessing layers for resizing and rescaling images
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),  # Resizing images to the specified size
    layers.Rescaling(1.0/256)  # Normalizing pixel values between 0 and 1
])

In [ ]:
# Data augmentation layers to randomly flip and rotate images during training
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),  # Randomly flip images horizontally and vertically
    layers.RandomRotation(0.2)  # Randomly rotate images by 20% to improve generalization
])

## 6. Model Definition and Customization

In [ ]:
# Defining the input shape of the model and number of output classes
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 5  # Assuming there are 5 classes

# Building a simple convolutional neural network (CNN) model
model = models.Sequential([
    resize_and_rescale,  # Apply resizing and rescaling to input images
    data_augmentation,  # Apply data augmentation during training
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),  # First convolutional layer
    layers.MaxPool2D((2, 2)),  # Max pooling layer to reduce spatial dimensions
    layers.Conv2D(32, (3, 3), activation='relu'),  # Second convolutional layer
    layers.MaxPool2D((2, 2)),  # Max pooling layer
    layers.Flatten(),  # Flatten the output for the fully connected layer
    layers.Dense(128, activation='relu'),  # Fully connected layer with 128 units
    layers.Dense(n_classes, activation='softmax')  # Output layer with softmax activation for classification
])

# Building the model with the specified input shape
model.build(input_shape=input_shape)

In [ ]:
# Displaying the model summary (layer details)
model.summary()

## 7. Model Compilation and Training

In [ ]:
from tensorflow.keras.optimizers import Adam

# Defining the optimizer with a small learning rate
optimizer = Adam(learning_rate=1e-5)  # Reducing learning rate for better convergence

# Compiling the model with an appropriate loss function and evaluation metric
model.compile(
    optimizer=optimizer,
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),  # Loss for multi-class classification
    metrics=['accuracy']  # Evaluate model performance using accuracy
)

In [ ]:
# Defining a callback to save the best model based on validation loss
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_checkpoint.keras',
    save_best_only=True,  # Save only the model with the lowest validation loss
    monitor='val_loss',  # Monitor the validation loss
    mode='min'  # Save the model when the validation loss is minimized
)

# Training the model on the training data, validating on the validation data
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    verbose=1,  # Display progress during training
    validation_data=val_ds,  # Use the validation data to monitor performance
    callbacks=[checkpoint_callback]  # Use the checkpoint callback to save the best model
)

In [ ]:
# Evaluating the model on the test dataset
score = model.evaluate(test_ds)

## 8. Model Saving and Loading

In [ ]:
# Saving the trained model to a specified directory
model_version = 1
save_dir = f"./models/{model_version}.keras"

# Checking if the directory exists and creating it if necessary
os.makedirs(os.path.dirname(save_dir), exist_ok=True)

# Saving the model to the specified directory
model.save(save_dir)

In [ ]:
# Loading the saved model for future use
model_version = 1
model_path = f"./models/{model_version}.keras"
model = tf.keras.models.load_model(model_path)

## 9. Prediction Function and Visualization

In [ ]:
# Function to make predictions on a single image and return the predicted class and confidence
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)  # Convert image to array
    img_array = tf.expand_dims(img_array, 0)  # Add a batch dimension
    
    predictions = model.predict(img_array)  # Make predictions
    
    # Get the predicted class and confidence
    predicted_class = class_names[np.argmax(predictions[0])]  # Class with the highest probability
    confidence = round(100 * (np.max(predictions[0])), 2)  # Confidence score in percentage
    
    return predicted_class, confidence

In [ ]:
# Visualizing predictions on the test dataset
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):  # Taking one batch from the test set
    for i in range(9):  # Display 9 images
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype('uint8'))  # Show the image
        
        # Get predicted and actual class labels with confidence scores
        predicted_class, confidence = predict(model, images[i])
        actual_class = class_names[labels[i]]
        
        # Display the predicted and actual class labels
        plt.title(f"Predicted: {predicted_class} ({confidence}%)\nActual: {actual_class}")
        plt.axis('off')  # Turn off axis